In [1]:
#deep embedded model
#Author John Xiang
%load_ext autotime
import pandas as pd
import numpy as np
import gc
from sklearn.metrics import roc_auc_score

from sklearn.preprocessing import LabelEncoder
def process_lable_encoder(col, train_df):
    le = LabelEncoder()
    le.fit(np.hstack(train_df[col]))
    train_df[col+str('_encoded')] = le.transform(train_df[col])
    
    #test_df[col] = le.transform(test_df[col])

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler , StandardScaler

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential , Model

from keras.layers import Input , Dense  , Flatten , concatenate
from keras.layers.embeddings import Embedding
from keras.callbacks import EarlyStopping
from keras import optimizers, regularizers, metrics
from keras.optimizers import SGD, Adagrad , Adam

C:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '

time: 2.63 s


In [3]:
from azure.datalake.store import core, lib, multithread
import pandas as pd
import pickle
token = lib.auth(tenant_id = 'fd3cd373-5dc9-4f59-9826-3aa2dd40d354'
               , username='John.xiang@inteliusanalytics.com'
               , password='XIANGlan960719.')
adl = core.AzureDLFileSystem(token, store_name='trugendatalake')
with adl.open(('LeadGen/Raw/Silver/AUTO/Train_data/x_train.pkl'), 'rb') as f:
    df = pickle.load(f)
print('x_train load done')
with adl.open(('LeadGen/Raw/Silver/AUTO/Train_data/y_train.pkl'), 'rb') as f:
    target = pickle.load(f)
print('y_train load done')
pd.options.display.float_format = '{:,.2f}'.format

x_train load done
y_train load done
time: 36.8 s


In [4]:
target['FULL_TERM_PREMIUM_COVERAGE'].describe()

count   8,142,138.00
mean          138.30
std           285.85
min        -1,325.45
25%            15.00
50%            36.00
75%           145.00
max        98,400.00
Name: FULL_TERM_PREMIUM_COVERAGE, dtype: float64

time: 516 ms


In [5]:
train_df=pd.concat([df,target],axis =1)

time: 1.58 s


In [6]:
num_cols = ['AGE_D1',
           'DEDUCTIBLE_AMOUNT',
           'DISTANCE_DRIVEN_ANNUALLY',
           'LIMIT_1',
           'VEHICLE_RATE_GROUP',
           'DRIVING_AGE',
           'VEHICLE_AGE',
           'DRIVERS_NUM',
           'FULL_TERM_PREMIUM_COVERAGE']

#target = ['FULL_TERM_PREMIUM_COVERAGE']

cat_cols = ['MULTICAR_DISCOUNT_INDICATOR',
 'COVERAGE_CODE_AUTOMOBILE',
 'COMPANY_CODE',
 'POSTAL_CODE',
 'VEHICLE_CODE1']

for col in num_cols:
    train_df[col] =pd.to_numeric(train_df[col],errors='coerce')

time: 635 ms


In [8]:
for cols in cat_cols:
    process_lable_encoder(cols,train_df)
print("label gen done")

label gen done
time: 3min 40s


In [14]:
encoded_cat_cols = ["COVERAGE_CODE_AUTOMOBILE_encoded",
"MULTICAR_DISCOUNT_INDICATOR_encoded",
"VEHICLE_CODE1_encoded",
"COMPANY_CODE_encoded",
"POSTAL_CODE_encoded"]

time: 999 µs


In [10]:
countDf = pd.DataFrame(columns = ['count'], data = train_df.count().values)
countDf['colName'] = train_df.count().index
countDf['percent'] = countDf['count']/len(train_df)
countDf.sort_values('percent',ascending = False)

,count,colName,percent
0,8142138,AGE_D1,1.00
10,8142138,VEHICLE_AGE,1.00
17,8142138,POSTAL_CODE_encoded,1.00
16,8142138,COMPANY_CODE_encoded,1.00
15,8142138,COVERAGE_CODE_AUTOMOBILE_encoded,1.00
14,8142138,MULTICAR_DISCOUNT_INDICATOR_encoded,1.00
13,8142138,FULL_TERM_PREMIUM_COVERAGE,1.00
12,8142138,DRIVERS_NUM,1.00
11,8142138,MULTICAR_DISCOUNT_INDICATOR,1.00
9,8142138,DRIVING_AGE,1.00


time: 13.7 s


In [20]:
def col_completness(data , col): 
        print('{}, completness = {}'.format(col, 1-(data[col].isnull().values.sum()/data.shape[0])))

        
def embedding_cols(df , cols, replace = True,  missing_string = 'nan_nan'):
    data = df.copy()
    embed_Dic = {}
    for col in cols: 
        print('{} is getting filled by integers'.format(col))
        uniques , rng = data[col].unique().tolist() , list(range(len(data[col].unique())))
        embed_dic = dict(zip(uniques , rng))
        embed_dic['nan-nan'] =  len(rng) +10
        if replace: 
            data.replace({col : embed_dic}, inplace = True)
        data = data.astype({col: int})
        embed_Dic[col] = embed_dic
    return data , embed_Dic       



def data_to_model_preparation( data , cat_cols , num_cols, shuffle = True): 
    if shuffle: 
        data = data.sample(frac=1).reset_index(drop=True)

    X_cat , Xy_num = data.loc[:, cat_cols].values , data.loc[:, num_cols].values
    #splt= int((data.shape[0]*train_split)//1)
    
    Xtrain_cat = X_cat[:,:]
    #Xtrain_cat, Xtest_cat = X_cat[:splt,:], X_cat[splt:,:]
    Xytrain_num = Xy_num[:,:]
    #Xytrain_num, Xytest_num = Xy_num[:splt,:], Xy_num[splt:,:]


    scaler = MinMaxScaler()
#     scaler = StandardScaler()
    scaler.fit(Xytrain_num)
    print(scaler.fit(Xytrain_num))
    Xytrain_num_scaled = scaler.transform(Xytrain_num)

    Xtrain_num , ytrain  = Xytrain_num_scaled[:, :-1] , Xytrain_num_scaled[:, -1]
    #Xtest_num , ytest  = Xytest_num_scaled[:, :-1] , Xytest_num_scaled[:, -1]
    
    ytrain.shape = (ytrain.shape[0] , 1)
    #ytest.shape = (ytest.shape[0], 1)
    if len(num_cols) == 1: 
        np.expand_dims(Xtrain_num, 1)
        #np.expand_dims(Xtest_num, 1)
    print('categoricalcolumns: ', cat_cols, '\n')
    print('Numerical columns: ', num_cols, '\n')
    print('Xtrain_cat.shape ={} , Xtrain_num.shape ={} , ytrain.shape ={} '.format(Xtrain_cat.shape , Xtrain_num.shape, ytrain.shape))
    #print('Xtrain_cat.shape ={} , Xtest_cat.shape ={}, Xtrain_num.shape ={}, Xtest_num.shape ={} , ytrain.shape ={} , ytest.shape ={} '.format(Xtrain_cat.shape , Xtest_cat.shape, Xtrain_num.shape, Xtest_num.shape , ytrain.shape , ytest.shape))

    return Xtrain_cat , Xtrain_num , ytrain  , scaler

def model_statistics(model, Xtst, ytst, scaler):

        mn  = scaler.mean_[-1] 
        scale  = scaler.scale_[-1] 
        
        preds = model.predict(Xtst)*scale+mn
        #preds = np.exp(preds*4)*y-0.9
        yt = ytst.copy()
        yt = yt*scale + mn
        #yt = np.exp(yt*4)*y-0.9
        yt.shape = (yt.shape[0], 1)
        

        mae = np.mean(np.abs(preds-yt))
        r2 = 1- (np.mean(np.square(preds-yt))/np.var(yt))
        indata_mae = np.mean(np.abs(yt-np.mean(yt)))
        model_MSE = np.mean(np.square(preds-yt))
        print('Model_MSE = {} ; target_VAR = {}'.format(model_MSE , np.var(yt)), '\n')
        print('inData_MAE = {} ; model_MAE = {} ; model_R_Square = {}'.format(indata_mae, mae, r2), '\n')
        return mae , r2 , indata_mae
    
def alpha(x):
    return int((x**0.25)*3)

time: 11 ms


In [136]:
scaler = StandardScaler()
scaler.fit(Xytrain_num)
print(scaler.fit(Xytrain_num))
#Xytrain_num_scaled = scaler.transform(Xytrain_num)

StandardScaler(copy=True, with_mean=True, with_std=True)
time: 5.93 s


In [ ]:
Xytrain_num

In [139]:
f = open(r"C:\Users\john.lan\Desktop\scaler.pkl","wb")
pickle.dump(Xytrain_num,f)
f.close()

time: 7.68 s


In [22]:
for col in df.columns:
    print(col, len(df[col].unique()))

AGE_D1 96
POSTAL_CODE 179456
COMPANY_CODE 55
COVERAGE_CODE_AUTOMOBILE 125
DEDUCTIBLE_AMOUNT 123
DISTANCE_DRIVEN_ANNUALLY 814
LIMIT_1 2018
VEHICLE_CODE1 6772
VEHICLE_RATE_GROUP 99
DRIVING_AGE 90
VEHICLE_AGE 107
MULTICAR_DISCOUNT_INDICATOR 2
DRIVERS_NUM 35
COVERAGE_CODE_AUTOMOBILE_encoded 125
MULTICAR_DISCOUNT_INDICATOR_encoded 2
VEHICLE_CODE1_encoded 6772
COMPANY_CODE_encoded 55
POSTAL_CODE_encoded 179456
time: 9.32 s


In [16]:
dim_dic = {col:alpha(len(train_df[col].unique())) for col in encoded_cat_cols}

time: 779 ms


In [17]:
dim_dic

{'COVERAGE_CODE_AUTOMOBILE_encoded': 10,
 'MULTICAR_DISCOUNT_INDICATOR_encoded': 3,
 'VEHICLE_CODE1_encoded': 27,
 'COMPANY_CODE_encoded': 8,
 'POSTAL_CODE_encoded': 61}

time: 3 ms


In [47]:
target.columns

Index(['FULL_TERM_PREMIUM_COVERAGE'], dtype='object')

time: 4.03 ms


In [48]:
train_df = pd.concat([df,target],axis =1)

time: 1.92 s


In [21]:
from keras import backend as K
vocab_size_dic = {}
for col in encoded_cat_cols: 
    vocab_size_dic[col] = len(train_df[col].unique())+10
    
max_length = 1

inputs_list = []
aux_list = []

for col in encoded_cat_cols: 
    
        inputs_cat= Input(shape =(1,), name = 'inputs_'+col)
        inputs_list.append(inputs_cat)
        outputs_cat = Embedding(vocab_size_dic[col] , dim_dic[col], input_length=max_length, 
                                name = 'embedding_'+col)(inputs_cat)
        flat = Flatten( name = 'flat_'+col)(outputs_cat)
        aux_list.append(flat)
        

inputs_num = Input(shape= (len(num_cols)-1, ), name = 'inputs_num')

model_inputs = inputs_list+[inputs_num]
aux = aux_list + [inputs_num]      
outputs = concatenate(aux)
outputs = Dense(100, activation = 'tanh')(outputs)
outputs = Dense(50, activation = 'tanh')(outputs)
outputs = Dense(20, activation = 'tanh')(outputs)#change to relu, last dense change to sigmod
outputs = Dense(5)(outputs)
model_outputs = Dense(1)(outputs)

model = Model(model_inputs  , model_outputs)

sgd = SGD(lr=0.01)
adagrad = Adagrad(lr=0.1, epsilon=1e-08, decay=0.0)
adam = optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

def r_square(y_true, y_pred):
    var = K.var(y_true)
    mse = K.mean(K.square(y_true - y_pred))
    return (1-mse/var)
model.compile(optimizer=adagrad, loss=['mse'],metrics = [r_square])

time: 1.09 s


In [130]:
train_df.FULL_TERM_PREMIUM_COVERAGE.describe()

count   8,142,138.00
mean          138.30
std           285.85
min        -1,325.45
25%            15.00
50%            36.00
75%           145.00
max        98,400.00
Name: FULL_TERM_PREMIUM_COVERAGE, dtype: float64

time: 829 ms


In [57]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8142138 entries, 90015 to 174538
Data columns (total 19 columns):
AGE_D1                                 float64
POSTAL_CODE                            object
COMPANY_CODE                           object
COVERAGE_CODE_AUTOMOBILE               object
DEDUCTIBLE_AMOUNT                      float64
DISTANCE_DRIVEN_ANNUALLY               float64
LIMIT_1                                float64
VEHICLE_CODE1                          object
VEHICLE_RATE_GROUP                     float64
DRIVING_AGE                            float64
VEHICLE_AGE                            float64
MULTICAR_DISCOUNT_INDICATOR            int64
DRIVERS_NUM                            float64
COVERAGE_CODE_AUTOMOBILE_encoded       int32
MULTICAR_DISCOUNT_INDICATOR_encoded    int64
VEHICLE_CODE1_encoded                  int32
COMPANY_CODE_encoded                   int32
POSTAL_CODE_encoded                    int32
FULL_TERM_PREMIUM_COVERAGE             float64
dtypes:

In [88]:
print(len(coverage_dict))
print(len(multicar_dict))
print(len(vehicle_dict))
print(len(company_dict))
print(len(postal_dict))

125
2
6772
55
179456
time: 2 ms


In [120]:
f = open(r"C:\Users\john.lan\Desktop\coverage_dict.pkl","wb")
pickle.dump(coverage_dict,f)
f.close()

time: 12 ms


In [121]:
f = open(r"C:\Users\john.lan\Desktop\multicar_dict.pkl","wb")
pickle.dump(multicar_dict,f)
f.close()

time: 9 ms


In [122]:
f = open(r"C:\Users\john.lan\Desktop\vehicle_dict.pkl","wb")
pickle.dump(vehicle_dict,f)
f.close()

time: 13 ms


In [123]:
f = open(r"C:\Users\john.lan\Desktop\company_dict.pkl","wb")
pickle.dump(company_dict,f)
f.close()

time: 9 ms


In [124]:
f = open(r"C:\Users\john.lan\Desktop\postal_dict.pkl","wb")
pickle.dump(postal_dict,f)
f.close()

time: 129 ms


In [ ]:
Xtrain_cat , Xtrain_num , ytrain  , scaler = data_to_model_preparation( data=train_df , 
                          cat_cols=encoded_cat_cols , num_cols= num_cols , shuffle = True )

for col in encoded_cat_cols:
    print(np.array_equal(train_df[col] , train_df[col].astype(int)))



Xtr = [Xtrain_cat[:,k] for k in range(Xtrain_cat.shape[1])] + [Xtrain_num[:, 0:]]
#Xts = [Xtest_cat[:,k] for k in range(Xtest_cat.shape[1])] + [Xtest_num[:, 0:]]

#early_stopping = EarlyStopping(monitor='val_loss', patience=10) ## put r2 and val_loss

history = model.fit(Xtr , ytrain , batch_size = 2048 , epochs = 40, verbose = 1 )


StandardScaler(copy=True, with_mean=True, with_std=True)
categoricalcolumns:  ['COVERAGE_CODE_AUTOMOBILE_encoded', 'MULTICAR_DISCOUNT_INDICATOR_encoded', 'VEHICLE_CODE1_encoded', 'COMPANY_CODE_encoded', 'POSTAL_CODE_encoded'] 

Numerical columns:  ['AGE_D1', 'DEDUCTIBLE_AMOUNT', 'DISTANCE_DRIVEN_ANNUALLY', 'LIMIT_1', 'VEHICLE_RATE_GROUP', 'DRIVING_AGE', 'VEHICLE_AGE', 'DRIVERS_NUM', 'FULL_TERM_PREMIUM_COVERAGE'] 

Xtrain_cat.shape =(8142138, 5) , Xtrain_num.shape =(8142138, 8) , ytrain.shape =(8142138, 1) 
True
True
True
True
True
Epoch 1/40
8142138/8142138 [==============================] - 542s 67us/step - loss: 0.3038 - r_square: 0.7263
Epoch 2/40
8142138/8142138 [==============================] - 535s 66us/step - loss: 0.1757 - r_square: 0.8534
Epoch 3/40
8142138/8142138 [==============================] - 537s 66us/step - loss: 0.1323 - r_square: 0.8941
Epoch 4/40
8142138/8142138 [==============================] - 533s 65us/step - loss: 0.1078 - r_square: 0.9167
Epoch 5/40
8142138/

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open(r"C:\Users\john.lan\Desktop\deep_embedding_training_4.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(r"C:\Users\john.lan\Desktop\deep_embedding_training_4.h5")
print("Saved model to disk")

In [127]:
mn  = scaler.mean_[-1] 
mn

138.30303879429067

time: 4.04 ms


In [131]:
scaler

StandardScaler(copy=True, with_mean=True, with_std=True)

time: 2.98 ms


285.8540882020679

time: 2.99 ms


In [128]:
scale  = scaler.scale_[-1]
scale

285.8540882020679

time: 2.99 ms


In [29]:
preds = model.predict(Xts)

print(1-(np.mean(np.square(preds - ytest))/np.var(ytest)), np.var(ytest))

# 0.9830499349825353 1.0002365561527946
# time: 21.7 s
# 0.825348875891685 1.0122958655371341
# time: 49.5 s

0.9674572023713434 0.9885359261314155
time: 7.2 s


In [133]:
_,_,_ = model_statistics(model, Xtst=Xtr, ytst= ytrain, scaler= scaler)

# Model_MSE = 0.01338294773984117 ; target_VAR = 0.7895514010379386 

# inData_MAE = 0.6252041419312583 ; model_MAE = 0.036451583305443305 ; model_R_Square = 0.9830499347829058 

# time: 21.3 s

Model_MSE = 3159.4740374503385 ; target_VAR = 81712.55974183559 

inData_MAE = 157.29299587755122 ; model_MAE = 17.279047452360434 ; model_R_Square = 0.9613342912346345 

time: 3min 25s


In [90]:
mn  = scaler.mean_[-1] 
scale  = scaler.scale_[-1] 
preditions = model.predict(Xts)*scale+mn

time: 31.4 s


In [91]:
preditions

array([[   3.2137299],
       [ 283.40942  ],
       [2357.35     ],
       ...,
       [   8.882996 ],
       [  41.726547 ],
       [  43.18853  ]], dtype=float32)

time: 3 ms


In [92]:
ytest*scale+mn

array([[  13.],
       [ 393.],
       [2638.],
       ...,
       [   0.],
       [  75.],
       [  72.]])

time: 35 ms
